In [78]:
import numpy as np

In [79]:
class HaloPoint:
    def __init__(self, ax, mass, color_decay, color1, color2='white'):
        self.halo_points_color1 = [ax.plot([], [], 'o', c=color1, markersize=0, alpha=0, markeredgecolor='none', zorder=3)[0] for _ in range(100)]
        self.halo_points_white = [ax.plot([], [], 'o', c=color2, markersize=0, alpha=0, markeredgecolor='none', zorder=4)[0] for _ in range(100)]
        self.mass = mass
        self.color_decay = color_decay
        
    def set_data(self, x, y):
        radius = np.linspace(2, 20, 100)
        for index, i in enumerate(radius):
            self.halo_points_color1[index].set_data([x], [y])
            self.halo_points_white[index].set_data([x], [y])
            
            self.halo_points_color1[index].set_markersize(self.mass*i)
            self.halo_points_color1[index].set_alpha(np.exp(-(i-2))**self.color_decay)
            
            self.halo_points_white[index].set_markersize(self.mass*i)
            self.halo_points_white[index].set_alpha(np.exp(-(i-2))**1)
        
    def get_artists(self):
        return self.halo_points_color1 + self.halo_points_white

In [80]:
# from matplotlib.animation import FuncAnimation
# import matplotlib.pyplot as plt
# 
# fig, ax = plt.subplots()
# ax.set_xlim(0, 10)  # Zum Beispiel
# ax.set_ylim(0, 10)  # Zum Beispiel
# 
# ax.grid(False)
# ax.axis('off')
# ax.set_xlim(-1, 10.5)
# fig.patch.set_facecolor('black')
# 
# # Beispiel-Daten (ersetzen Sie diese durch Ihre eigenen Daten)
# t = np.linspace(0, 2 * np.pi, 4000)  # Zeitpunkte
# x = 5 + 3 * np.cos(t)  # x-Koordinaten
# y = 5 + 3 * np.sin(t)
# 
# halo_point = HaloPoint(ax, 2, .8, color1='dodgerblue')
# 
# def update(frame):
#     # Setzen Sie die Daten für den Halo-Punkt
#     halo_point.set_data(x[frame], y[frame])
#     return halo_point.get_artists()
# 
# ani = FuncAnimation(fig, update, frames=100, blit=True, interval=10)
# plt.show()